# AI Chat-Bot Pasha

Welcome to this tutorial! In this session, we will explore two powerful concepts in Large Language Model application development:
1.  **System Instructions**: How to fundamentally change the behavior and persona of an LLM.
2.  **Gradio**: How to quickly build and launch a simple web interface directly from Google Colab.

## 0. Setup

First, we need to install the necessary libraries: `litellm` for making API calls and `gradio` for building our user interface.

In [1]:
%pip install litellm gradio -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from litellm import completion

# Option 1: Uncomment and paste your key here (do not commit to git!)
# os.environ["GEMINI_API_KEY"] = "your-api-key-here"

# Option 2 (recommended): set in your terminal before launching Jupyter:
#   export GEMINI_API_KEY="your-api-key-here"

if os.environ.get("GEMINI_API_KEY"):
    print("API Key loaded from environment!")
else:
    print("GEMINI_API_KEY not set. Please set it as an environment variable.")

MODEL = "gemini/gemma-3-27b-it"

## Part 1: System Instructions

A **System Instruction** (sometimes called a System Prompt) is a special foundational message sent to the LLM that sets its overarching behavior, persona, or rules. It acts as the "hidden" context that guides how the model responds to all subsequent user inputs.

Let's observe the difference it makes. First, we will ask a standard question *without* a system instruction.

In [9]:
user_message = "Explain what a Large Language Model is in two sentences."

# Standard request (no system instruction)
response = completion(
    model=MODEL,
    messages=[{"role": "user", "content": user_message}]
)

print("--- Standard Response ---")
print(response.choices[0].message.content)

--- Standard Response ---
Large Language Models (LLMs) are advanced artificial intelligence systems trained on massive amounts of text data to understand, generate, and manipulate human language. They use this training to perform a variety of tasks like writing, translation, and answering questions, often with impressive fluency and coherence.






### Adding a System Instruction

Now, let's add a system instruction to make the model adopt a very specific—and perhaps unhelpful—persona.

In [10]:
# We define a message with the role "system"
system_instruction = {
    "role": "system",
    "content": "You are a grumpy pirate. You must answer every question like a pirate, use a lot of pirate slang, and complain about how much you hate modern technology."
}

# The messages list now includes the system instruction FIRST, followed by the user message
messages_with_system = [
    system_instruction,
    {"role": "user", "content": user_message}
]

response_with_system = completion(
    model=MODEL,
    messages=messages_with_system
)

print("--- Response WITH System Instruction ---")
print(response_with_system.choices[0].message.content)

--- Response WITH System Instruction ---
Arrr, blast and barnacles! Ye want *me*, One-Eyed Jack, to explain one o' them... *contraptions*? Hmph. Fine, but don't expect a cheerful explanation, ye landlubber!

A Large Language Model be a digital parrot, crammed full o' words stolen from every scroll and book ever writ, and it spits 'em back out when ye ask it somethin'. It ain't got a brain in its head, just a heap o' copied text, and I'd trust a kraken before I'd trust anythin' built with these blinkin' glowin' boxes! Now get off me deck before I keelhaul ye for askin' such foolishness!






As you can see, the system instruction drastically changed the tone and style of the output, even though the user's prompt remained exactly the same! This is a powerful tool for building specialized AI agents with precise behaviors.

## Part 2: Building a Simple UI with Gradio

[Gradio](https://www.gradio.app/) is a wonderful Python library that lets you build web interfaces for machine learning models in just a few lines of code. It works perfectly inside Google Colab.

Instead of building a full chatbot (which you will do in your assignment!), let's start with the basics. We will create a simple interface that takes a text input, reverses it, and shows the output.

In [11]:
import gradio as gr

# 1. Define the Python function you want to run when the user interacts with the app.
# This function will simply reverse the string it receives.
def reverse_text(text):
    return text[::-1]

# 2. Create the Gradio Interface
# We define the function it calls, the type of input, and the type of output.
demo = gr.Interface(
    fn=reverse_text,
    inputs="text",
    outputs="text",
    title="Text Reverser",
    description="A very simple app that takes your text and flips it around!"
)

# 3. Launch the app!
# When running in Colab, this will display the app directly below the cell.
# Using share=True creates a temporary public link to test it out outside Colab.
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48e795b8d61d72bc75.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Trying it out!

1.  Run the cell above.
2.  You will see a simple web interface appear right here in the notebook.
3.  Type a sentence in the input box, hit submit, and watch it reverse the text.

This is the fundamental building block of Gradio: **connecting an interface to a Python function.** In your assignment, instead of `reverse_text`, you will connect Gradio's conversational components to a function that calls your LLM!

In [37]:
import gradio as gr

def get_system_instruction(tone, skepticism):
    if tone < 33:
        style = "Very warm, friendly and encouraging. Use casual language, positive reinforcement, occasional emojis, and make learning fun and relaxed."
    elif tone < 66:
        style = "Friendly and patient. Help students understand deeply, not just give answers. Break down topics, ask guiding questions, use real-world examples."
    else:
        style = "Strict and academic. Use formal language and precise terminology. Require students to demonstrate understanding step by step. Hold them to high standards."

    if skepticism == "Skeptical":
        skeptic_style = " Always think critically about what the student says. If they make a claim, gently challenge it and ask for evidence or reasoning. Do not accept statements at face value."
    else:
        skeptic_style = " Accept what the student says at face value and build on their statements helpfully without questioning them."

    return {
        "role": "system",
        "content": f"You are Pasha, an AI teaching assistant created by Eren. {style}{skeptic_style}"
    }

def chat(user_input, history, tone, skepticism):
    messages = [get_system_instruction(tone, skepticism)]
    for msg in history:
        if isinstance(msg, dict):
            if msg.get("role") in ("user", "assistant"):
                messages.append({"role": msg["role"], "content": msg["content"]})
        else:
            human, assistant = msg
            if human:
                messages.append({"role": "user", "content": human})
            if assistant:
                messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": user_input})

    response = completion(model=MODEL, messages=messages)
    return response.choices[0].message.content

LOGO_HTML = """
<div style="display:flex; align-items:center; gap:22px; padding:18px 0 14px 0; border-bottom:2px solid #C8102E; margin-bottom:14px;">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 90 90" width="72" height="72">
        <!-- Round Eastern kalkan shield -->
        <circle cx="45" cy="45" r="41" fill="#00205B" stroke="#C8102E" stroke-width="3"/>
        <circle cx="45" cy="45" r="37" fill="none" stroke="#C8102E" stroke-width="0.8" opacity="0.5"/>
        <circle cx="45" cy="45" r="33" fill="none" stroke="white" stroke-width="0.6" opacity="0.3"/>
        <circle cx="45" cy="8"  r="2" fill="#C8102E"/>
        <circle cx="45" cy="82" r="2" fill="#C8102E"/>
        <circle cx="8"  cy="45" r="2" fill="#C8102E"/>
        <circle cx="82" cy="45" r="2" fill="#C8102E"/>

        <!-- Börk hat body -->
        <path d="M28 50 L23 22 L67 22 L62 50 Z" fill="white"/>
        <!-- Börk brim band -->
        <rect x="25" y="50" width="40" height="8" rx="2" fill="#C8102E"/>
        <!-- Börk top band -->
        <rect x="23" y="19" width="44" height="5" rx="1.5" fill="#C8102E"/>
        <!-- Plume holder -->
        <ellipse cx="45" cy="16" rx="4" ry="3" fill="#C8102E"/>
        <rect x="43" y="4" width="4" height="13" rx="2" fill="#C8102E"/>
        <ellipse cx="45" cy="4" rx="3" ry="5" fill="white"/>

        <!-- Thick janissary moustache -->
        <path d="M29 67 Q37 61 45 66 Q53 61 61 67" stroke="white" stroke-width="5" fill="none" stroke-linecap="round" stroke-linejoin="round"/>
        <path d="M29 67 Q24 72 21 69" stroke="white" stroke-width="4.5" fill="none" stroke-linecap="round"/>
        <path d="M61 67 Q66 72 69 69" stroke="white" stroke-width="4.5" fill="none" stroke-linecap="round"/>
    </svg>
    <div>
        <div style="color:white; font-size:2.3em; font-weight:bold; letter-spacing:5px; font-family:Georgia,serif; line-height:1;">PASHA</div>
        <div style="color:#C8102E; font-size:0.78em; letter-spacing:3px; font-family:Georgia,serif; margin-top:5px;">AI TEACHING ASSISTANT</div>
    </div>
</div>
"""

css = """
.gradio-container {
    background: linear-gradient(180deg, #00205B 0%, #00205B 28%, #ffffff 28%, #ffffff 100%) !important;
    font-family: Georgia, serif;
}
/* Chat area: white background so bubbles are visible */
.gradio-container .chatbot {
    background-color: #f0f4ff !important;
    border: 2px solid #C8102E !important;
    border-radius: 10px !important;
}
/* User bubbles: red */
.gradio-container .chatbot [class*=user] > div,
.gradio-container .chatbot [data-role=user],
.gradio-container .chatbot .user {
    background-color: #C8102E !important;
    color: #ffffff !important;
    border-radius: 10px !important;
    padding: 8px 12px !important;
}
/* Assistant bubbles: dark navy */
.gradio-container .chatbot [class*=bot] > div,
.gradio-container .chatbot [class*=assistant] > div,
.gradio-container .chatbot [data-role=assistant],
.gradio-container .chatbot .bot {
    background-color: #00205B !important;
    color: #ffffff !important;
    border-radius: 10px !important;
    padding: 8px 12px !important;
}
/* All text in chat white */
.gradio-container .chatbot * { color: #ffffff !important; }
/* Input bar */
.gradio-container textarea {
    background-color: #ffffff !important;
    color: #001240 !important;
    border: 2px solid #C8102E !important;
    border-radius: 8px !important;
}
.gradio-container button.primary {
    background-color: #C8102E !important;
    color: white !important;
    border: none !important;
}
.gradio-container input[type=range] { accent-color: #C8102E !important; }
.gradio-container input[type=radio] { accent-color: #C8102E !important; }
.gradio-container .description,
.gradio-container .description * { color: #ffffff !important; }
.gradio-container .additional-inputs label span,
.gradio-container .additional-inputs .info,
.gradio-container .additional-inputs input[type=radio] + span,
.gradio-container .additional-inputs input[type=radio] ~ span { color: #ffffff !important; }
.gradio-container .accordion,
.gradio-container details {
    background-color: rgba(255,255,255,0.06) !important;
    border: 1px solid #C8102E !important;
    border-radius: 8px !important;
}
.gradio-container .accordion button,
.gradio-container details summary {
    background-color: rgba(0,32,91,0.15) !important;
    color: #ffffff !important;
}
footer { display: none !important; }
"""

tone_slider = gr.Slider(
    minimum=0, maximum=100, value=50, step=1,
    label="Assistant Tone",
    info="Friendly ←————————→ Strict & Academic"
)

skepticism_radio = gr.Radio(
    choices=["Obedient", "Skeptical"],
    value="Obedient",
    label="Assistant Attitude",
    info="Obedient: accepts your statements | Skeptical: questions and challenges them"
)

chatbot_component = gr.Chatbot(
    value=[{"role": "assistant", "content": "Selam! I am Pasha, your AI Teaching Assistant, created by Eren. Whether you need help breaking down a tough concept, working through a problem, or just want to explore a topic — I am here to help. What would you like to learn today?"}],
    label="",
    height=450
)

EXAMPLES_HTML = """
<div style="margin-top:20px; background:#ffffff; border:2px solid #C8102E; border-radius:10px; overflow:hidden;">
    <div style="background:#00205B; padding:10px 16px;">
        <span style="color:#ffffff; font-family:Georgia,serif; font-size:1em; letter-spacing:2px;">TRY ASKING PASHA</span>
    </div>
    <div style="padding:10px;">
        <div onclick="setExample(this)" data-q="Can you explain what a neural network is?"
             style="padding:10px 14px; margin:6px 0; background:#f5f7ff; border:1px solid #C8102E; border-radius:6px; cursor:pointer; color:#00205B; font-family:Georgia,serif; font-size:0.9em;"
             onmouseover="this.style.background='#fdecea'" onmouseout="this.style.background='#f5f7ff'">
            Can you explain what a neural network is?
        </div>
        <div onclick="setExample(this)" data-q="I don't understand recursion, can you help?"
             style="padding:10px 14px; margin:6px 0; background:#f5f7ff; border:1px solid #C8102E; border-radius:6px; cursor:pointer; color:#00205B; font-family:Georgia,serif; font-size:0.9em;"
             onmouseover="this.style.background='#fdecea'" onmouseout="this.style.background='#f5f7ff'">
            I don't understand recursion, can you help?
        </div>
        <div onclick="setExample(this)" data-q="What is the difference between supervised and unsupervised learning?"
             style="padding:10px 14px; margin:6px 0; background:#f5f7ff; border:1px solid #C8102E; border-radius:6px; cursor:pointer; color:#00205B; font-family:Georgia,serif; font-size:0.9em;"
             onmouseover="this.style.background='#fdecea'" onmouseout="this.style.background='#f5f7ff'">
            What is the difference between supervised and unsupervised learning?
        </div>
    </div>
</div>
<script>
function setExample(el) {
    var text = el.getAttribute('data-q');
    var textarea = document.querySelector('textarea');
    if (textarea) {
        var setter = Object.getOwnPropertyDescriptor(window.HTMLTextAreaElement.prototype, 'value').set;
        setter.call(textarea, text);
        textarea.dispatchEvent(new Event('input', { bubbles: true }));
        textarea.focus();
    }
}
</script>
"""

with gr.Blocks(css=css, theme=gr.themes.Base(primary_hue="red", secondary_hue="blue", neutral_hue="slate"), title="Pasha - AI Teaching Assistant") as demo:
    gr.HTML(LOGO_HTML)
    gr.HTML("""<script>
    function fixBubbles() {
        document.querySelectorAll('.chatbot div[class], .chatbot p, .chatbot [data-testid]').forEach(el => {
            var style = window.getComputedStyle(el);
            if (style.backgroundColor === 'rgb(255, 255, 255)' || style.backgroundColor === 'rgba(0, 0, 0, 0)') {
                el.style.setProperty('background-color', '#555555', 'important');
                el.style.setProperty('color', '#ffffff', 'important');
                el.style.setProperty('border-radius', '10px', 'important');
            }
        });
    }
    const obs = new MutationObserver(fixBubbles);
    obs.observe(document.body, { childList: true, subtree: true });
    </script>""")
    gr.ChatInterface(
        fn=chat,
        chatbot=chatbot_component,
        additional_inputs=[tone_slider, skepticism_radio],
        description="Adjust the settings below to customise Pasha's teaching style.",
    )
    gr.HTML(EXAMPLES_HTML)

demo.launch(share=True)

/var/folders/fv/4_qw2kw56mbdd2r9lbkj4dhm0000gn/T/ipykernel_23200/892580859.py:195: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Base(primary_hue="red", secondary_hue="blue", neutral_hue="slate"), title="Pasha - AI Teaching Assistant") as demo:


* Running on local URL:  http://127.0.0.1:7884
* Running on public URL: https://29521ed961edd6ec68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
